In [18]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

Loading files as Pandas dataframes, they can be easily be dumped into the sqlite database. Since Pandas dataframes must fit on the RAM, this might not work for very large datasets.

In [19]:
air_reserve     = pd.read_csv('input/air_reserve.csv',parse_dates=['visit_datetime','reserve_datetime'])
hpg_reserve     = pd.read_csv('input/hpg_reserve.csv',parse_dates=['visit_datetime','reserve_datetime'])
air_store_info = pd.read_csv('input/air_store_info.csv')
hpg_store_info = pd.read_csv('input/hpg_store_info.csv')
store_relation = pd.read_csv('input/store_id_relation.csv')
date_info = pd.read_csv('input/date_info.csv',parse_dates=['calendar_date'])
air_visit = pd.read_csv('input/air_visit_data.csv',parse_dates=['visit_date'])
sample_submission = pd.read_csv('input/sample_submission.csv')

Open a connection to sqlite file and dump the dataframes as individual tables.

In [22]:
conn = sqlite3.connect('input/inventory.db')
air_reserve.to_sql(name='air_reserve',if_exists='replace',con=conn)
hpg_reserve.to_sql(name='hpg_reserve',if_exists='replace',con=conn)
air_store_info.to_sql(  'air_store_info',if_exists='replace',con=conn)
hpg_store_info.to_sql('hpg_store_info',if_exists='replace',con=conn)
store_relation.to_sql('store_relation',if_exists='replace',con=conn)
date_info.to_sql('date_info',if_exists='replace',con=conn)
air_visit.to_sql('air_visit',if_exists='replace',con=conn)
conn.commit()

In [ ]:
sql = '''
create table air_res_store as 
select 
t1.*,
t2.air_genre_name,
t2.air_area_name 
from 
air_reserve t1 
join 
air_store_info t2 
on 
t1.air_store_id=t2.air_store_id
'''

In [ ]:
sql='''
create table air_res_store1 as
select
air_store_id,
reserve_visitors,
air_genre_name,
air_area_name,
date(visit_datetime) visit_date,
time(visit_datetime) visit_hour ,
date(reserve_datetime) reserve_date, 
time(reserve_datetime) reserve_hour
from air_res_store t1;
'''


In [ ]:
sql = '''
create table air_res_store2 as
select 
t1.*, 
t2.day_of_week, 
t2.holiday_flg 
from 
air_res_store1 t1 
join 
date_info t2 
on t1.visit_date = date(t2.calendar_date);'''

In [ ]:
sql='''
create table air_res_store3 as
select 
t1.*, 
t2.visitors as day_visitors 
from 
air_res_store2 t1 
join 
air_visit t2 
on 
t1.air_store_id = t2.air_store_id 
and t1.visit_date = date(t2.visit_date);
'''